In [229]:
import numpy as np
from keras.datasets import imdb
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import json

In [230]:
def preprocess(X):
    tokenizer = RegexpTokenizer(r'\w+')
    X = [tokenizer.tokenize(x) for x in X]
    stop_words = set(stopwords.words('english'))
    X_stop=[]
    for words in X:
        words = [w for w in words if not w in stop_words]
        X_stop.append(words)
    porter = PorterStemmer()
    X_stem=[]
    for words in X_stop:
        stemmed = [porter.stem(word) for word in words]
        X_stem.append(stemmed)
    return X_stem

In [231]:
import csv
X_train = []
X_test = []
Y_train = []
with open('train.csv') as f:
    read = csv.reader(f)
    next(read, None)
    for (item,label,sentence) in read:
        X_train.append(sentence)
        Y_train.append(label)
with open('test.csv') as f:
    read = csv.reader(f)
    next(read, None)
    for (item,sentence) in read:
        X_test.append(sentence)

In [232]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)
Y_train = list(map(int, Y_train))

In [174]:
result = [len(x) for x in X_train_pad]
print(sum(result)/len(result))
print(max(result))

100.0
100


In [165]:
from gensim.models import Word2Vec
model = Word2Vec(X_train+X_test, min_count=1)

KeyboardInterrupt: 

In [109]:
weights = model.wv.syn0
np.save(open('embeddings', 'wb'), weights)
vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
with open('vocab', 'w') as f:
    f.write(json.dumps(vocab))


C:\Users\simrat.s\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [166]:
with open('vocab', 'r') as f:
    data = json.loads(f.read())
word2idx = data
idx2word = dict([(v, k) for k, v in data.items()])

In [167]:
weights = np.load(open('embeddings', 'rb'))

In [206]:
weights.shape[1]

100

In [169]:
max_words = 100
#X_train = seqnce.pad_sequences(X_train, maxlen=max_words)
#X_test = sequence.pad_sequences(X_test, maxlen=max_words)
word2idx['apl']

32842

In [170]:
X_train_idx = [[word2idx[text] for text in x] for x in X_train]
X_test_idx = [[word2idx[text] for text in x] for x in X_test]

In [173]:
X_train_pad = sequence.pad_sequences(X_train_idx, maxlen=max_words)
X_test_pad = sequence.pad_sequences(X_test_idx, maxlen=max_words)

In [220]:
layer = Embedding(input_dim=weights.shape[0],
                      output_dim=weights.shape[1],
                      weights=[weights], input_length=100)
model = Sequential()
model.add(layer)
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 100)          23698500  
_________________________________________________________________
flatten_3 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 250)               2500250   
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 251       
Total params: 26,199,001
Trainable params: 26,199,001
Non-trainable params: 0
_________________________________________________________________
None


In [233]:
X_final_train = X_train_pad[0:-20000]
Y_final_train = Y_train[0:-20000]
X_val = X_train_pad[-20000:]
Y_val = Y_train[-20000:]

In [234]:
model.fit(X_final_train, Y_final_train, validation_data=(X_val, Y_val), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_val, Y_val, verbose=2)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 79989 samples, validate on 20000 samples
Epoch 1/2
 - 369s - loss: 8.9768 - acc: 0.4425 - val_loss: 9.5435 - val_acc: 0.4079
Epoch 2/2
 - 370s - loss: 8.9901 - acc: 0.4422 - val_loss: 9.5435 - val_acc: 0.4079
Accuracy: 40.79%
